# Imports 

In [116]:
!pip install ccxt pandas numpy

In [117]:
!pip install ta-lib  

In [118]:
!pip install ta-lib talib-binary 
!pip install pandas-ta    

ERROR: Could not find a version that satisfies the requirement talib-binary (from versions: none)
ERROR: No matching distribution found for talib-binary


In [119]:
!pip uninstall logging

In [120]:
!pip install asyncio websocket-client
# !pip install sqlite3 
# !pip install logging  

In [121]:
!pip list

Package                   Version
------------------------- -----------
aiodns                    3.5.0
aiohappyeyeballs          2.6.1
aiohttp                   3.12.15
aiosignal                 1.4.0
altair                    5.5.0
asttokens                 3.0.0
async-timeout             5.0.1
asyncio                   4.0.0
attrs                     25.3.0
backcall                  0.2.0
blinker                   1.9.0
build                     1.3.0
cachetools                6.2.0
ccxt                      4.5.2
certifi                   2025.8.3
cffi                      1.17.1
charset-normalizer        3.4.3
click                     8.1.8
colorama                  0.4.6
comm                      0.2.3
contourpy                 1.3.0
cryptography              45.0.6
cycler                    0.12.1
dash                      3.2.0
debugpy                   1.8.16
decorator                 5.2.1
executing                 2.2.0
Flask                     3.1.2
fonttools             

In [122]:
!pip install plotly dash          
!pip install matplotlib seaborn   
!pip install streamlit 

In [123]:
!pip install python-dotenv       
!pip install schedule             
!pip install requests aiohttp     

^C


In [ ]:
!pip install scikit-learn

# Else

In [2]:
import os 
from dotenv import load_dotenv

In [3]:
load_dotenv()

False

In [1]:
class Configur():
    EXCHANGE  = "binance"
    SYMBOL = "BTC/USDT"
    TIMEFRAME = "1h"


    API_KEY = ""
    SECRET_KEY = ""
    SANDBOX = ""

    INIT_BALANCE = 10000
    RISK = 0.01
    STOP_LOSS = 0.02
    PROFIT_TAKE = 0.04
    # Strategy Parameters
    RSI_PERIOD = 14
    RSI_OVERSOLD = 30
    RSI_OVERBOUGHT = 70
    EMA_SHORT = 20
    EMA_LONG = 50
    
    # Execution Settings
    MIN_TRADES_FOR_METRICS = 50
    UPDATE_INTERVAL = 60  # seconds
    
    # Logging
    LOG_FILE = 'trades.csv'
    DB_FILE = 'trading_bot.db'



In [2]:
import ccxt
import pandas as pd
import numpy as np
import asyncio
import time
from datetime import datetime
from Config import Configs

In [3]:
import ccxt
import pandas as pd
import numpy as np
import asyncio
import time
from datetime import datetime
from Config import Configs

class Manage_data:
    def __init__(self):
        self.exchange = self._init_exchange()
        self.current_price = 0
        self.ohlcv_data = pd.DataFrame()
        
    def _init_exchange(self):
        exchange_class = getattr(ccxt, 'binance')
        exchange = exchange_class({
            'apiKey': Configs.API_KEY,
            'secret': Configs.SECRET_KEY,
            'sandbox': Configs.SANDBOX,
            'enableRateLimit': True,
        })
        return exchange      
    
    def fetch_ohlcv(self, limit =800):
        try:
            ohlcv = self.exchange.fetch_ohlcv(
                "BTC/USDT",
                "1h",
                limit =limit
            )
            df = pd.DataFrame(ohlcv,columns=['timestamp','open','high','low','close','volume'])
            df.set_index('timestamp', inplace=True)
            self.ohlcv_data = df
            self.current_price = df['close'].iloc[-1]
            return df
        except Exception as e:
            print(e)
            return None
    def get_current_price(self):
        try :
            ticker = self.exchange.fetch_ticker("BTC/USDT")
            self.current_price = ticker['last']
            return self.current_price
        except Exception as e:
            print(f" problem in get_current_price {e}")
            return self.current_price
    def get_balance(self):
        try:
            balance = self.exchange.fetch_balance()
            return  balance['USDT']['free'] if 'USDT' in balance else Configur.INIT_BALANCE
        except Exception as e:
            print( f" problem in get_balance {e}")
            return Configur.INIT_BALANCE

In [4]:
data = Manage_data()
df = data.fetch_ohlcv()
print(df)

                    open       high        low      close    volume
timestamp                                                          
1753959600000  118763.42  119007.81  117899.30  118364.79   5.72396
1753963200000  118342.94  118645.10  117951.97  118626.18  12.58421
1753966800000  118626.18  118692.08  117711.32  118573.28  13.16674
1753970400000  118568.63  119071.21  117273.45  117913.19  13.17513
1753974000000  117913.18  121615.28   30000.00  118297.31  30.31667
...                  ...        ...        ...        ...       ...
1756821600000  111133.80  112174.80  110785.82  111367.99  21.42454
1756825200000  111367.99  111500.00  110638.03  110829.14  15.55030
1756828800000  110829.14  132536.10  110473.69  110772.00  10.99981
1756832400000  110771.99  111050.00  109931.30  110914.01  21.87393
1756836000000  110914.00  110914.01  108592.00  110565.61  12.96195

[800 rows x 5 columns]


In [5]:
import numpy as np 
import pandas as pd 
import talib


def fet_Ext(df):
    df['price_change'] =  df['close'].pct_change()#15 min change 
    df['price_change_5'] =  df['close'].pct_change(5)#change in 75 mins
    price_range=(df['high'].rolling(14).max()-df['low'].rolling(14).min())
    df['pos_range_prc'] = df['close']-df['low'].rolling(14).min()/price_range
    
    df['rsi'] = talib.RSI(df['close'].values,14)
    df['rsi_oversold'] = (df['rsi'] < 30).astype(int)# stock sold too much will rise
    df['rsi_overbought'] = (df['rsi'] > 80).astype(int)# stock bought too much will fall

    #moving AVGS
    df['smal_10'] = df['close'].rolling(10).mean()# Short term mean
    df['smal_20'] = df['close'].rolling(20).mean()# Long term mean
    df['price_above_smal_10'] = (df['close']>df['smal_10']).astype(int)
    df['price_above_smal_20'] = (df['close']>df['smal_20']).astype(int)
    df['sma_trend'] = (df['smal_10'] > df['smal_20']).astype(int)

    
    df['volume_avg'] = df['volume'].rolling(20).mean()
    df['volume_high'] = (df['volume'] > df['volume_avg'] * 1.5).astype(int)



    df['volatility'] = df['price_change'].rolling(10).std()
    df['high_volatility'] = (df['volatility'] > df['volatility'].rolling(50).mean()).astype(int)

    df['future_return'] = df['price_change'].shift(-1)
    df['target'] = np.where(df['future_return'] > 0.002, 1, 
                            np.where(df['future_return'] < -0.002, -1, 0))
    
    return df.fillna(0)

In [6]:
df = fet_Ext(df)

In [7]:
df

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1753959600000,118763.42,119007.81,117899.30,118364.79,5.72396,0.000000,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,0.002208,1
1753963200000,118342.94,118645.10,117951.97,118626.18,12.58421,0.002208,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.000446,0
1753966800000,118626.18,118692.08,117711.32,118573.28,13.16674,-0.000446,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.005567,-1
1753970400000,118568.63,119071.21,117273.45,117913.19,13.17513,-0.005567,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,0.003258,1
1753974000000,117913.18,121615.28,30000.00,118297.31,30.31667,0.003258,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,0.002031,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756821600000,111133.80,112174.80,110785.82,111367.99,21.42454,0.002107,0.008555,111367.813529,63.678173,0,...,109781.4635,1,1,1,23.902590,0,0.007874,1,-0.004838,-1
1756825200000,111367.99,111500.00,110638.03,110829.14,15.55030,-0.004838,0.005319,110828.963529,58.386775,0,...,109862.0385,1,1,1,23.602820,0,0.008074,1,-0.000516,0
1756828800000,110829.14,132536.10,110473.69,110772.00,10.99981,-0.000516,0.009650,110771.823529,57.837930,0,...,109957.7730,1,1,1,22.534124,0,0.008076,1,0.001282,0


In [8]:
feat_df = fet_Ext(df)

In [9]:
len(df)

800

# Train Model

In [13]:
from Config import Configur

ImportError: cannot import name 'Configur' from 'Config' (c:\Users\Com\OneDrive\Documents\GitHub\Crypto_bot\Config.py)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pickle


MIN_DATA_POINTS = 300
class RandomForestStrat:
    def __init__(self):
        self.name = "Random_forest_classifier"
        self.model = RandomForestClassifier(
            n_estimators = 50,
            max_depth=8,
            random_state= 42,
            class_weight='balanced'
        )
        self.features = fet_Ext
        self.feature_list = [
        'price_change', 'price_change_5', 'pos_range_prc',
        'rsi', 'rsi_oversold', 'rsi_overbought',
        'price_above_smal_10', 'price_above_smal_20', 'sma_trend',
        'volume_high', 'high_volatility']
        self.is_trained = False


    def Train(self,df):
        df = fet_Ext(df)
        feature_columns =  self.feature_list+ ['target']
        data = df[feature_columns].dropna()
        if len(data) < MIN_DATA_POINTS:
            #More data points are required
            return False 
        X = data[self.feature_list]
        Y = data['target']

        self.model.fit(X,Y)

        y_pred = self.model.predict(X)
        accuracy = accuracy_score(Y,y_pred)

        print(f"Accuracy:{accuracy}")
        with open("Weighted_model.pkl", "wb") as f:
            pickle.dump(self.model, f)

        self.is_trained = True
        return True
    
    def predict(self, df):
        """Make prediction"""
        if not self.is_trained:
            return 'hold', 0.0
        
        # Create features for latest data
        df = self.features(df)
        latest_data = df[self.feature_list].iloc[-1:].fillna(0)
        
        # Get prediction and confidence
        prediction = self.model.predict(latest_data)[0]
        probabilities = self.model.predict_proba(latest_data)[0]
        confidence = max(probabilities)
        
        # Convert to signal
        if confidence > 0.8:
            if prediction == 1:
                return 'buy', confidence
            elif prediction == -1:
                return 'sell', confidence
        
        return 'hold', confidence

In [14]:
mod = RandomForestStrat()

In [15]:
model  = mod.Train(df)

Accuracy:0.885


# Test It out

In [10]:
df[:1]

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1753959600000,118763.42,119007.81,117899.3,118364.79,5.72396,NaN,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,0.002208,1


In [11]:
df.tail()

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1756821600000,111133.80,112174.80,110785.82,111367.99,21.42454,0.002107,0.008555,111367.813529,63.678173,0,...,109781.4635,1,1,1,23.902590,0,0.007874,1,-0.004838,-1
1756825200000,111367.99,111500.00,110638.03,110829.14,15.55030,-0.004838,0.005319,110828.963529,58.386775,0,...,109862.0385,1,1,1,23.602820,0,0.008074,1,-0.000516,0
1756828800000,110829.14,132536.10,110473.69,110772.00,10.99981,-0.000516,0.009650,110771.823529,57.837930,0,...,109957.7730,1,1,1,22.534124,0,0.008076,1,0.001282,0
1756832400000,110771.99,111050.00,109931.30,110914.01,21.87393,0.001282,0.019468,110913.833529,58.872665,0,...,110113.4330,1,1,1,23.176437,0,0.008059,1,-0.003141,-1
1756836000000,110914.00,110914.01,108592.00,110565.61,12.96195,-0.003141,-0.005113,110565.433529,55.287742,0,...,110219.4130,1,1,1,23.350800,0,0.008144,1,NaN,0


In [12]:
df[-1:]

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1756836000000,110914.0,110914.01,108592.0,110565.61,12.96195,-0.003141,-0.005113,110565.433529,55.287742,0,...,110219.413,1,1,1,23.3508,0,0.008144,1,NaN,0


In [13]:
import pickle

with open("Weighted_model.pkl", "rb") as f:
    models = pickle.load(f)
test_df = df[-100:]
test_df = test_df.drop(['target'], axis=1)
feature_list = [
'price_change', 'price_change_5', 'pos_range_prc',
'rsi', 'rsi_oversold', 'rsi_overbought',
'price_above_smal_10', 'price_above_smal_20', 'sma_trend',
'volume_high', 'high_volatility']
X = test_df[feature_list]
dictions  = models.predict(X)

In [14]:
from sklearn.metrics import accuracy_score
res_df = pd.DataFrame()
res_df['results'] = dictions
accuracy = accuracy_score(res_df['results'],df['target'][-100:])
print(accuracy)

0.89


In [15]:
df['target'].value_counts()

target
 0    448
 1    181
-1    171
Name: count, dtype: int64

In [16]:
dictions[-20:]

array([ 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  1,  1, -1,  0,
        0,  0,  0])

In [17]:
side = {-1:"sell",0:"hold",1:"buy"}

In [18]:
df.index = pd.to_datetime(df.index)

In [19]:
df

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1970-01-01 00:29:13.959600,118763.42,119007.81,117899.30,118364.79,5.72396,NaN,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,0.002208,1
1970-01-01 00:29:13.963200,118342.94,118645.10,117951.97,118626.18,12.58421,0.002208,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,-0.000446,0
1970-01-01 00:29:13.966800,118626.18,118692.08,117711.32,118573.28,13.16674,-0.000446,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,-0.005567,-1
1970-01-01 00:29:13.970400,118568.63,119071.21,117273.45,117913.19,13.17513,-0.005567,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,0.003258,1
1970-01-01 00:29:13.974000,117913.18,121615.28,30000.00,118297.31,30.31667,0.003258,NaN,NaN,NaN,0,...,NaN,0,0,0,NaN,0,NaN,0,0.002031,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:29:16.821600,111133.80,112174.80,110785.82,111367.99,21.42454,0.002107,0.008555,111367.813529,63.678173,0,...,109781.4635,1,1,1,23.902590,0,0.007874,1,-0.004838,-1
1970-01-01 00:29:16.825200,111367.99,111500.00,110638.03,110829.14,15.55030,-0.004838,0.005319,110828.963529,58.386775,0,...,109862.0385,1,1,1,23.602820,0,0.008074,1,-0.000516,0
1970-01-01 00:29:16.828800,110829.14,132536.10,110473.69,110772.00,10.99981,-0.000516,0.009650,110771.823529,57.837930,0,...,109957.7730,1,1,1,22.534124,0,0.008076,1,0.001282,0


In [20]:
%load_ext autoreload
%autoreload 2

In [ ]:
import asyncio
import pickle
from datetime import datetime
from GetData import Manage_data
from Manage_orders import ExecutionEngine
from Logger import TradeLogger
from Visualizer import Visualizerr
from Config import Configs
from FeatureExtract import Features
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import dash
from dash import dcc, html, Input, Output
import threading


class CryptoTradingBot:
    def __init__(self, df=None):
        self.data_manager = Manage_data()
        self.execution_engine = ExecutionEngine()
        self.logger = TradeLogger()
        self.visualizer = Visualizerr()
        self.last_timestamp = None
        self.running = False
        self.Extractor = Features()
        self.feature_list  =[
        'price_change', 'price_change_5', 'pos_range_prc',
        'rsi', 'rsi_oversold', 'rsi_overbought',
        'price_above_smal_10', 'price_above_smal_20', 'sma_trend',
        'volume_high', 'high_volatility']
        self.position = None
        self.test_df = df  # premade DataFrame
        with open("Weighted_model.pkl", "rb") as f:
            self.models = pickle.load(f)

    def run_bot(self):
        """Main bot execution loop"""
        self.running = True
        print(f"Starting crypto trading bot for {Configs.SYMBOL}")
        print(f"Strategy: Random Forest ")
        print(f"Initial Balance: ${Configs.INIT_BALANCE}")
        print("-" * 50)
        

        for i in range( 100, len(self.test_df)-1):  # start after enough candles for indicators
            try:
                df = self.test_df.iloc[: i+1].copy()

                clean_df = df.dropna()
                Ext_Features = self.Extractor.fet_Ext(clean_df)
                feat_list = self.Extractor.feature_list
                clean_df = Ext_Features[feat_list]
                
                self.signal = self.models.predict(clean_df)
                
                self.side = {-1: "sell", 0: "hold", 1: "buy"}

                current_price = df["close"].iloc[-1]
                current_time = df.index[-1]
                # print(self.side[self.signal[-1]])

                # # Check exit conditions
                self.execution_engine.check_exit_conditions(current_price, current_time)

                # # Execute new signal
                self.execution_engine.execute_signal(self.side[self.signal[-1]], current_price, current_time)

                # Log completed trades
                if self.execution_engine.trades:
                    for trade in self.execution_engine.trades:
                        if trade not in getattr(self, 'logged_trades', []):
                            self.logger.log_trade(trade)
                            if not hasattr(self, 'logged_trades'):
                                self.logged_trades = []
                            self.logged_trades.append(trade)

                # Display status
                self.display_status(df, self.signal, current_price)

            except Exception as e:
                print(f"Error in backtest loop: {e}")
                break
    def display_status(self, df, signal, price):
        """Display current bot status"""
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        rsi = df['rsi'].iloc[-1] if 'rsi' in df.columns else 0
        
        print(f"\n[{current_time}] Price: ${price:.2f} | RSI: {rsi:.1f} | Signal: {signal}")
        
        if self.execution_engine.position:
            pos = self.execution_engine.position
            unrealized_pnl = pos.calculate_pnl(price)
            print(f"Position: {pos.side.upper()} | Size: {pos.size:.4f} | Entry: ${pos.entry_price:.2f} | Unrealized P&L: ${unrealized_pnl:.2f}")
        
        print(f"Balance: ${self.execution_engine.balance:.2f} | Total Trades: {len(self.execution_engine.trades)}")
    def start_dashboard(self):
        """Start web dashboard in separate thread"""
        def run_dash():
            # Setup callbacks for live updates
            @self.visualizer.app.callback(
                [Output('price-chart', 'figure'),
                 Output('equity-chart', 'figure'),
                 Output('metrics-display', 'children')],
                [Input('interval-component', 'n_intervals')]
            )
            def update_dashboard(n):
                # Get latest data
                
                # if 'rsi' not in df.columns and len(df) > 0:
                #     df = self.strategy.calculate_indicators(df)
                
                # Create charts
                price_fig = self.visualizer.create_price_chart(
                    self.test_df, 
                    self.execution_engine.trades, 
                    self.data_manager.current_price
                )
                
                equity_fig = self.visualizer.create_equity_curve(
                    self.execution_engine.equity_curve
                )
                
                # Create metrics display
                if len(self.execution_engine.trades) > 0:
                    metrics = self.logger.calculate_metrics(self.execution_engine.trades)
                    metrics_div = html.Div([
                        html.H3("Performance Metrics"),
                        html.P(f"Win Rate: {metrics['win_rate']:.1f}%"),
                        html.P(f"Total P&L: ${metrics['total_pnl']:.2f}"),
                        html.P(f"Total Trades: {metrics['total_trades']}"),
                        html.P(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}"),
                        html.P(f"Max Drawdown: ${metrics['max_drawdown']:.2f}")
                    ])
                else:
                    metrics_div = html.Div([html.H3("Waiting for trades...")])
                
                return price_fig, equity_fig, metrics_div
            
            self.visualizer.app.run(debug=False, host='127.0.0.1', port=8050)
        
        dashboard_thread = threading.Thread(target=run_dash)
        dashboard_thread.daemon = True
        dashboard_thread.start()
        print("Dashboard started at http://127.0.0.1:8050")


[2025-09-03 00:25:03,259] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

In [36]:
bot  = CryptoTradingBot()


bot.start_dashboard()

Dashboard started at http://127.0.0.1:8050


[2025-09-03 00:24:23,641] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

In [ ]:
import importlib, Visualizer
importlib.reload(Visualizer)

from Visualizer import Visualizerr


[2025-09-03 00:17:13,294] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

Visualizer got df columns: RangeIndex(start=0, stop=0, step=1)
Shape: (0, 0)


[2025-09-03 00:17:16,293] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

Visualizer got df columns: RangeIndex(start=0, stop=0, step=1)
Shape: (0, 0)


[2025-09-03 00:17:18,294] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

...

Visualizer got df columns: RangeIndex(start=0, stop=0, step=1)
Shape: (0, 0)


[2025-09-03 00:18:28,306] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

Visualizer got df columns: RangeIndex(start=0, stop=0, step=1)
Shape: (0, 0)


[2025-09-03 00:18:33,295] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

Visualizer got df columns: RangeIndex(start=0, stop=0, step=1)
Shape: (0, 0)


In [25]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'price_change',
       'price_change_5', 'pos_range_prc', 'rsi', 'rsi_oversold',
       'rsi_overbought', 'smal_10', 'smal_20', 'price_above_smal_10',
       'price_above_smal_20', 'sma_trend', 'volume_avg', 'volume_high',
       'volatility', 'high_volatility', 'future_return', 'target'],
      dtype='object')

[2025-09-03 00:09:55,667] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

In [ ]:
# Load or create your test DataFrame
# df = pd.read_csv("your_test_data.csv", index_col=0, parse_dates=True)

# Pass it into the bot
bot = CryptoTradingBot(df=df)
# Start dashboard
bot.start_dashboard()

# Wait a moment for dashboard to start
time.sleep(2)

# Run trading bot
# bot.run_bot()

Dashboard started at http://127.0.0.1:8050


[2025-09-03 00:13:01,428] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

Visualizer got df columns: RangeIndex(start=0, stop=0, step=1)
Shape: (0, 0)


[2025-09-03 00:13:06,424] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

Visualizer got df columns: RangeIndex(start=0, stop=0, step=1)
Shape: (0, 0)


...

Visualizer got df columns: RangeIndex(start=0, stop=0, step=1)
Shape: (0, 0)


[2025-09-03 00:16:43,290] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

Visualizer got df columns: RangeIndex(start=0, stop=0, step=1)
Shape: (0, 0)


[2025-09-03 00:16:48,289] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\site-packages\dash\dash.py", line 1494, in dispatch
    response_data = ctx.run(partial_func)
  File "c:\Users\Com\anaconda3\envs\krypto_trader\lib\

Visualizer got df columns: RangeIndex(start=0, stop=0, step=1)
Shape: (0, 0)


In [33]:
from Visualizer import Visualizerr 
import os, inspect,importlib
importlib.reload(Visualizer)
# Where Python is importing your Visualizer from
print("Visualizer is loaded from:", Visualizer.__file__)

print(inspect.getsource(Visualizerr.create_price_chart))

Visualizer is loaded from: c:\Users\Com\OneDrive\Documents\GitHub\Crypto_bot\Visualizer.py
    def create_price_chart(self, df, trades, current_price):
        """Create price chart with trade markers"""
        fig = make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.03,
            subplot_titles=('Price Chart', 'RSI'),
            row_width=[0.7, 0.3]
        )
        
        # Candlestick chart
        fig.add_trace(
            go.Candlestick(
                x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                name='Price'
            ),
            row=1, col=1
        )
        
        # EMAs
        if 'ema_short' in df.columns:
            fig.add_trace(
                go.Scatter(x=df.index, y=df['ema_short'], name='EMA 20', line=dict(color='orange')),
                row=1, col=1
            )
            fig.add

In [154]:
import time
from datetime import datetime
from enum import Enum

class OrderStatus(Enum):
    PENDING = "pending"
    FILLED  = "filled"
    CANCELLED   = "cancelled"

class Trade :
    def __init__(self,side,entry_price,size,timestamp):
        self.side  =  side
        self.entry_price= entry_price
        self.exit_price = None
        self.size = size
        self.enrty_time  = timestamp
        self.exit_time = None
        self.pnl = 0
        self.status= OrderStatus.PENDING
        self.stop_loss = None
        self.take_profit = None

    def set_exit_levels(self):
        if self.side =='buy':
            self.stop_loss = self.enrty_price * (1-Configur.STOP_LOSS)
            self.take_profit  = self.enrty_price *(1+Configur.PROFIT_TAKE)
        else:
            self.stop_loss = self.enrty_price * (1+Configur.STOP_LOSS)
            self.take_profit  = self.enrty_price *(1-Configur.PROFIT_TAKE)

    def calculate_pnl(self,exit_price):
        if self.side  == 'buy':
            self.pnl = (exit_price - self.enrty_price)*self.size
        else :
            self.pnl = (self.enrty_price - exit_price)*self.size
        return self.pnl
    
class ExecutionEngine:
    def __init__(self):
        self.data_manager = Manage_data
        self.balance  = Configur.INIT_BALANCE
        self.position = None
        self.trades  = []
        self.equity_curve = [Configur.INIT_BALANCE]
    
    def execute_signal(self,signal,price,timestamp):
        if signal == 'hold':
            return None
        
        if self.position and signal != self.position.side:
            self.close_position(price, timestamp)

        
        if not self.position and signal in ['buy', 'sell']:
            return self.open_position(signal, price, timestamp)
        
        return None
    
    def open_position(self, side, price, timestamp):
        """Open a new position"""
        from strategy import TradingStrategy
        strategy = TradingStrategy()
        
        size = strategy.calculate_position_size(self.balance, price)
        if size * price < 10:  # Minimum order size
            return None
            
        trade = Trade(side, price, size, timestamp)
        trade.set_exit_levels()
        self.position = trade
        
        print(f"OPENED {side.upper()} position: {size:.4f} @ ${price:.2f}")
        return trade
    
    def close_position(self, price, timestamp, reason="signal"):
        """Close current position"""
        if not self.position:
            return None
            
        self.position.exit_price = price
        self.position.exit_time = timestamp
        self.position.status = OrderStatus.FILLED
        
        pnl = self.position.calculate_pnl(price)
        self.balance += pnl
        self.equity_curve.append(self.balance)
        
        self.trades.append(self.position)
        
        print(f"CLOSED {self.position.side.upper()} position: @ ${price:.2f} | P&L: ${pnl:.2f} | Reason: {reason}")
        
        closed_trade = self.position
        self.position = None
        return closed_trade
    
    def check_exit_conditions(self, current_price, timestamp):
        """Check stop loss and take profit conditions"""
        if not self.position:
            return
            
        if self.position.side == 'buy':
            if current_price <= self.position.stop_loss:
                self.close_position(current_price, timestamp, "stop_loss")
            elif current_price >= self.position.take_profit:
                self.close_position(current_price, timestamp, "take_profit")
        else:  # sell
            if current_price >= self.position.stop_loss:
                self.close_position(current_price, timestamp, "stop_loss")
            elif current_price <= self.position.take_profit:
                self.close_position(current_price, timestamp, "take_profit")





In [155]:
import pandas as pd
import sqlite3
import json
from datetime import datetime

class TradeLogger:
    def __init__(self):
        self.trades_df = pd.DataFrame()
        self.init_database()
        
    def init_database(self):
        """Initialize SQLite database"""
        conn = sqlite3.connect(Configur.DB_FILE)
        conn.execute('''
            CREATE TABLE IF NOT EXISTS trades (
                id INTEGER PRIMARY KEY,
                side TEXT,
                entry_price REAL,
                exit_price REAL,
                size REAL,
                entry_time TEXT,
                exit_time TEXT,
                pnl REAL,
                status TEXT,
                stop_loss REAL,
                take_profit REAL
            )
        ''')
        conn.close()
    
    def log_trade(self, trade):
        """Log completed trade"""
        trade_data = {
            'side': trade.side,
            'entry_price': trade.entry_price,
            'exit_price': trade.exit_price,
            'size': trade.size,
            'entry_time': trade.entry_time.isoformat(),
            'exit_time': trade.exit_time.isoformat() if trade.exit_time else None,
            'pnl': trade.pnl,
            'status': trade.status.value,
            'stop_loss': trade.stop_loss,
            'take_profit': trade.take_profit
        }
        
        # Save to CSV
        df = pd.DataFrame([trade_data])
        if not os.path.exists(Configur.LOG_FILE):
            df.to_csv(Configur.LOG_FILE, index=False)
        else:
            df.to_csv(Configur.LOG_FILE, mode='a', header=False, index=False)
        
        # Save to database
        conn = sqlite3.connect(Configur.DB_FILE)
        df.to_sql('trades', conn, if_exists='append', index=False)
        conn.close()
        
        print(f"Trade logged: {trade.side} | P&L: ${trade.pnl:.2f}")
    
    def calculate_metrics(self, trades):
        """Calculate trading performance metrics"""
        if len(trades) == 0:
            return {}
            
        pnls = [t.pnl for t in trades]
        wins = [p for p in pnls if p > 0]
        
        metrics = {
            'total_trades': len(trades),
            'winning_trades': len(wins),
            'win_rate': len(wins) / len(trades) * 100,
            'total_pnl': sum(pnls),
            'avg_win': np.mean(wins) if wins else 0,
            'avg_loss': np.mean([p for p in pnls if p < 0]) if any(p < 0 for p in pnls) else 0,
            'profit_factor': sum(wins) / abs(sum([p for p in pnls if p < 0])) if any(p < 0 for p in pnls) else float('inf'),
            'sharpe_ratio': np.mean(pnls) / np.std(pnls) if np.std(pnls) != 0 else 0,
            'max_drawdown': self._calculate_max_drawdown(pnls)
        }
        
        return metrics
    
    def _calculate_max_drawdown(self, pnls):
        """Calculate maximum drawdown"""
        cumulative = np.cumsum(pnls)
        running_max = np.maximum.accumulate(cumulative)
        drawdown = cumulative - running_max
        return abs(min(drawdown)) if len(drawdown) > 0 else 0